### Copy and compress video files
- as video files are created, if they are at least 1 hr long, upload them

In [1]:
import ffmpeg
import numpy as np
from tqdm.auto import tqdm
import subprocess
from IPython.display import clear_output
import os
import shutil
from time import time, sleep
from pathlib2 import Path
import pathlib2 
import os
from tqdm import tqdm

In [2]:
# settings
crf = 15 # 0 = lossless, 51 = worst

In [3]:
# mirror flip bottom cameras 
cameras_to_mirror = []#["FB", "BB"]

In [4]:
base_path_local = Path("/media/dattalab/starling_rec/recordings/")
base_path_save= Path('/n/groups/datta/tim_sainburg/datasets/3D_keypoints/compressed/') 
subfolder = '23-08-14-chronic_recordings'

In [5]:
# '/home/dattalab/JARVIS/sync_flir_azure_9_27/recordings/Recording_2022-09-27T122406'
# folder with recordings
video_folder = csv_folder = base_path_local / subfolder
# folder to save recordings folder
output_folder = base_path_save / subfolder

In [6]:
if not os.path.exists('/n/groups/datta/tim_sainburg'):
    raise ValueError('must mount /n/groups/datta')

In [7]:
save_folder = output_folder /video_folder.stem

In [8]:
def ensure_dir(file_path):
    """ create a safely nested folder
    """
    if type(file_path) == str:
        if "." in os.path.basename(os.path.normpath(file_path)):
            directory = os.path.dirname(file_path)
        else:
            directory = os.path.normpath(file_path)
        if not os.path.exists(directory):
            try:
                os.makedirs(directory)
            except FileExistsError as e:
                # multiprocessing can cause directory creation problems
                print(e)
    elif type(file_path) == pathlib2.PosixPath:
        # if this is a file
        if len(file_path.suffix) > 0:
            file_path.parent.mkdir(parents=True, exist_ok=True)
        else:
            file_path.mkdir(parents=True, exist_ok=True)

In [9]:
def copy_metadata(metadata_to_copy, triggerdata_to_copy):
    # copy all metadata (metadata.csv and triggerdata.csv)

    for csv_file in tqdm(metadata_to_copy + triggerdata_to_copy):
        # get the name to save for the output folder
        number_of_parents_to_base = np.where([i == csv_folder for i in list(csv_file.parents)])[0][0]
        sub_folders = csv_file.parts[len(csv_file.parts)-1-number_of_parents_to_base:-1]
        csv_out = output_folder
        for i in sub_folders:
            csv_out = csv_out / i
        csv_out = csv_out / csv_file.name

        ensure_dir(csv_out)

        if csv_out.exists():
            print('Video already exists, skipping \n\t {}'.format('/'.join(video.parts[-3:])))

            #if delete_local_video:
                #os.remove(csv_file)

            continue

        else:
            shutil.copyfile(csv_file, csv_out)

In [10]:
def copy_videos(videos_to_convert):
    for video in tqdm(videos_to_convert):
        try:
            # get the name to save for the output folder
            number_of_parents_to_base = np.where([i == video_folder for i in list(video.parents)])[0][0]
            sub_folders = video.parts[len(video.parts)-1-number_of_parents_to_base:-1]
            video_out = output_folder
            for i in sub_folders:
                video_out = video_out / i
            video_out = video_out / video.name

            ensure_dir(video_out)

            if video_out.exists():
                print('Video already exists, skipping \n\t {}'.format('/'.join(video.parts[-3:])))

                if delete_local_video:
                    os.remove(video)

                continue

            print('converting {}'.format('/'.join(video.parts[-3:])))
            #continue
            # https://www.programcreek.com/python/example/117479/ffmpeg.output

            flip_video = video.stem in cameras_to_mirror

            ffmpeg_command = (
                ffmpeg
                .input(video.as_posix())

            )
            if flip_video:
                ffmpeg_command = ffmpeg_command.video.hflip()
            ffmpeg_command = (
                ffmpeg_command.output(
                    video_out.as_posix(),
                    vcodec = "libx265", 
                    crf = crf, 
                    vtag="hvc1", 
                    preset="ultrafast"
                )
            )
            out, _ = ffmpeg_command.run()


            # if out is successful 
            if delete_local_video:
                os.remove(video)

        except KeyError:
            # delete bad video file
            if video_out.exists():
                os.remove(video_out)
            break

        except Exception:
            # delete bad video file
            if video_out.exists():
                os.remove(video_out)

            clear_output(wait=True)
            print('BACKUP FAILED {}'.format(video))
            continue

In [11]:
def cull_files(files, time_before_upload_s):
    seconds_since_last_modified = [time() - i.stat().st_mtime for i in files]
    files = [
                i for i, j in zip(
                    files, 
                    seconds_since_last_modified
                ) if j > time_before_upload_s
            ]
    return files

In [12]:
# how long since the file has been last modified should we wait to back it up
time_before_upload_s = 60*60*2
delete_local_video = False

In [13]:
ensure_dir(save_folder)

In [14]:
while True:
    # pause for 10 seconds
    
    # get a list of the files that need uploaded
    videos_to_convert = list(video_folder.glob('**/*.avi'))
    videos_to_convert = cull_files(videos_to_convert, time_before_upload_s)
    
    metadata_to_copy = list(csv_folder.glob('**/metadata.csv'))
    triggerdata_to_copy = list(csv_folder.glob('**/triggerdata.csv'))
    metadata_to_copy = cull_files(metadata_to_copy, time_before_upload_s)
    triggerdata_to_copy = cull_files(triggerdata_to_copy, time_before_upload_s)
    
    print(len(videos_to_convert), len(metadata_to_copy), len(triggerdata_to_copy))
    
    # copy data
    if (len(metadata_to_copy) > 0) or (len(triggerdata_to_copy) > 0):
        copy_metadata(metadata_to_copy, triggerdata_to_copy)
    
    if len(videos_to_convert) > 0:
        copy_videos(videos_to_convert)
    
    sleep(10)


0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0


KeyboardInterrupt: 